In [396]:
import numpy as np
import pandas as pd
from scipy.special import softmax

In [397]:
tickers = ['AAPL', 'GOOGL', 'MSFT', 'META', 'AMZN']
df_sims = {t: pd.read_csv(f"../price_sim/{t}_sim.csv") for t in tickers}

In [398]:
df_sim = df_sims['AAPL']
np.corrcoef(df_sim['pred_delta_ratio'], df_sim['real_delta_ratio'])[0, 1], \
np.corrcoef(df_sim['pred_delta'], df_sim['real_delta'])[0, 1]

(0.2673152497690316, 0.2618983974743353)

In [399]:
def sigmoid(x, k=1):
    return np.array(1 / (1 + np.exp(-k * x)))

In [400]:
def arctan(x, k=1):
    return np.array(np.arctan(k * x) / (np.pi / 2))

In [401]:
def sharpe_ratio(capital, risk_free=0.0):
    returns = np.diff(capital) / capital[:-1]
    excess = np.mean(returns - risk_free)
    std_dev = np.std(returns)
    return (excess / std_dev) * np.sqrt(252)

In [402]:
def multi_preprocess(df_sims, pool):
    length = len(df_sims[pool[0]])
    real_price = np.column_stack([df_sims[t]['real_price'] for t in pool] + [np.ones(length)])
    pred_delta = np.column_stack([df_sims[t]['pred_delta'] for t in pool] + [np.zeros(length)])
    pred_ratio = np.column_stack([df_sims[t]['pred_delta'] / df_sims[t]['real_price'].shift(1) for t in pool] + [np.zeros(length)])
    return pred_ratio, pred_delta, real_price

In [403]:
def trade(ratios, prices):
    capital = []
    currency = [1.0]
    position = [0.0]
    for ratio, price in zip(ratios[1:], prices[:-1]):
        money = currency[-1] + position[-1] * price
        assert money >= 0, "Bankruptcy"
        capital.append(money)
        nxt_cur = money * (1 - ratio)
        nxt_pos = money * ratio / price
        currency.append(nxt_cur)
        position.append(nxt_pos)
    money = currency[-1] + position[-1] * prices[-1]
    capital.append(money)

    return pd.DataFrame({
        'ratios': ratios,
        'prices': prices,
        'capital': capital,
        'currency': currency,
        'position': position
    })

In [404]:
def multi_trade(ratios, prices):
    capital = []
    position = [np.eye(prices.shape[1])[-1]]
    for ratio, price in zip(ratios[1:], prices[:-1]):
        if not capital:
            money = 1.0
        else:
            money = (position[-1] * price).sum()
        capital.append(money)
        assert money >= 0, "Bankruptcy"
        nxt_pos = money * ratio / price
        position.append(nxt_pos)
    money = (position[-1] * prices[-1]).sum()
    capital.append(money)
    
    return pd.DataFrame({
        'ratios': np.sum(ratios[:, :-1], axis=1),
        'prices': np.mean(prices[:, :-1], axis=1),
        'capital': capital,
        'position': position
    })

In [405]:
annual_risk_free = 0.04
daily_risk_free = (1+annual_risk_free) ** (1/252) - 1
daily_risk_free

0.0001556498627912628

In [406]:
ratios1, prices1 = np.ones(len(df_sim), dtype=float), df_sim['real_price'].to_numpy()
df_trade1 = trade(ratios1, prices1)
ratios2, prices2 = sigmoid(df_sim['pred_delta'] / df_sim['real_price'].shift(1), 5 * df_sim['real_price'].mean()), df_sim['real_price'].to_numpy()
df_trade2 = trade(ratios2, prices2)
ratios3, prices3 = arctan(df_sim['pred_delta'] / df_sim['real_price'].shift(1), 1 * df_sim['real_price'].mean()), df_sim['real_price'].to_numpy()
df_trade3 = trade(ratios3, prices3)

In [407]:
print(f"Always hold stock: (Sharpe){sharpe_ratio(df_trade1['capital'], daily_risk_free):.2f}, (Yield){df_trade1['capital'].iloc[-1]:.2f}")
print(f"Single stock (no short selling): (Sharpe){sharpe_ratio(df_trade2['capital'], daily_risk_free):.2f}, (Yield){df_trade2['capital'].iloc[-1]:.2f}")
print(f"Single stock (short selling): (Sharpe){sharpe_ratio(df_trade3['capital'], daily_risk_free):.2f}, (Yield){df_trade3['capital'].iloc[-1]:.2f}")

Always hold stock: (Sharpe)1.37, (Yield)1.75
Single stock (no short selling): (Sharpe)3.32, (Yield)2.70
Single stock (short selling): (Sharpe)3.86, (Yield)2.53


In [408]:
pred_ratio, pred_delta, real_price = multi_preprocess(df_sims, tickers)
ratios4, prices4 = softmax(pred_ratio * 5 * real_price.mean(), axis=1), real_price
df_trade4 = multi_trade(ratios4, prices4)

In [409]:
prices5 = real_price[:, :-1]
ratios5 = np.full(prices5.shape, 1/len(tickers))
df_trade5 = multi_trade(ratios5, prices5)

In [410]:
avg_delta, avg_price = np.mean(pred_delta[:, :-1], axis=1), np.mean(real_price[:, :-1], axis=1)
ratios6, prices6 = sigmoid(avg_delta / np.roll(avg_price, 1), 5 * avg_price.mean()), avg_price
df_trade6 = trade(ratios6, prices6)

In [411]:
avg_delta, avg_price = np.mean(pred_delta[:, :-1], axis=1), np.mean(real_price[:, :-1], axis=1)
ratios7, prices7 = arctan(avg_delta / np.roll(avg_price, 1), 1 * avg_price.mean()), avg_price
df_trade7 = trade(ratios7, prices7)

In [412]:
atan_ratio = arctan(pred_ratio, 1 * real_price.mean())
base_ratio = 1/atan_ratio.shape[1]
ratios8, prices8 = (atan_ratio - atan_ratio.mean(axis=1, keepdims=True)) * (1-base_ratio) + base_ratio, real_price
df_trade8 = multi_trade(ratios8, prices8)

In [413]:
print(f"Fixed Portfolio (always hold): (Sharpe){sharpe_ratio(df_trade5['capital'], daily_risk_free):.2f}, (Yield){df_trade5['capital'].iloc[-1]:.2f}")
print(f"Fixed Portfolio (no short selling): (Sharpe){sharpe_ratio(df_trade6['capital'], daily_risk_free):.2f}, (Yield){df_trade6['capital'].iloc[-1]:.2f}")
print(f"Dynanmic Portfolio (no short selling): (Sharpe){sharpe_ratio(df_trade4['capital'], daily_risk_free):.2f}, (Yield){df_trade4['capital'].iloc[-1]:.2f}")
print(f"Fixed Portfolio (short selling): (Sharpe){sharpe_ratio(df_trade7['capital'], daily_risk_free):.2f}, (Yield){df_trade7['capital'].iloc[-1]:.2f}")
print(f"Dynanmic Portfolio (short selling): (Sharpe){sharpe_ratio(df_trade8['capital'], daily_risk_free):.2f}, (Yield){df_trade8['capital'].iloc[-1]:.2f}")

Fixed Portfolio (always hold): (Sharpe)1.99, (Yield)2.27
Fixed Portfolio (no short selling): (Sharpe)4.08, (Yield)3.57
Dynanmic Portfolio (no short selling): (Sharpe)4.71, (Yield)11.14
Fixed Portfolio (short selling): (Sharpe)4.67, (Yield)3.19
Dynanmic Portfolio (short selling): (Sharpe)6.46, (Yield)31.60
